# Challenge 2: Sentiment Analysis

In this challenge we will learn sentiment analysis and practice performing sentiment analysis on Twitter tweets. 

## Introduction

Sentiment analysis is to *systematically identify, extract, quantify, and study affective states and subjective information* based on texts ([reference](https://en.wikipedia.org/wiki/Sentiment_analysis)). In simple words, it's to understand whether a person is happy or unhappy in producing the piece of text. Why we (or rather, companies) care about sentiment in texts? It's because by understanding the sentiments in texts, we will be able to know if our customers are happy or unhappy about our products and services. If they are unhappy, the subsequent action is to figure out what have caused the unhappiness and make improvements.

Basic sentiment analysis only understands the *positive* or *negative* (sometimes *neutral* too) polarities of the sentiment. More advanced sentiment analysis will also consider dimensions such as agreement, subjectivity, confidence, irony, and so on. In this challenge we will conduct the basic positive vs negative sentiment analysis based on real Twitter tweets.

NLTK comes with a [sentiment analysis package](https://www.nltk.org/api/nltk.sentiment.html). This package is great for dummies to perform sentiment analysis because it requires only the textual data to make predictions. For example:

```python
>>> from nltk.sentiment.vader import SentimentIntensityAnalyzer
>>> txt = "Ironhack is a Global Tech School ranked num 2 worldwide.   Our mission is to help people transform their careers and join a thriving community of tech professionals that love what they do."
>>> analyzer = SentimentIntensityAnalyzer()
>>> analyzer.polarity_scores(txt)
{'neg': 0.0, 'neu': 0.741, 'pos': 0.259, 'compound': 0.8442}
```

In this challenge, however, you will not use NLTK's sentiment analysis package because in your Machine Learning training in the past 2 weeks you have learned how to make predictions more accurate than that. The [tweets data](https://www.kaggle.com/kazanova/sentiment140) we will be using today are already coded for the positive/negative sentiment. You will be able to use the Naïve Bayes classifier you learned in the lesson to predict the sentiment of tweets based on the labels.

## Conducting Sentiment Analysis

### Loading and Exploring Data

The dataset we'll be using today is located in the lab directory named `Sentiment140.csv.zip`. You need to unzip it into a `.csv` file. Then in the cell below, load and explore the data.

*Notes:* 

* The dataset was downloaded from [Kaggle](https://www.kaggle.com/kazanova/sentiment140). We made a slight change on the original data so that each column has a label.

* The dataset is huuuuge (1.6m tweets). When you develop your data analysis codes, you can sample a subset of the data (e.g. 20k records) so that you will save a lot of time when you test your codes.

In [1]:
import pandas as pd
import re

from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.sentiment.vader import SentimentIntensityAnalyzer

In [2]:
# your code here
col_name=['target','id','date', 'flag', 'user', 'text']
emotions_complete = pd.read_csv('emotions.csv', encoding='latin-1', names=col_name)

In [3]:
emotions_complete.sample(20000).target.value_counts()

4    10013
0     9987
Name: target, dtype: int64

In [4]:
emotions = emotions_complete.sample(20000)

In [5]:
emotions.target.value_counts()

4    10023
0     9977
Name: target, dtype: int64

### Prepare Textual Data for Sentiment Analysis

Now, apply the functions you have written in Challenge 1 to your whole data set. These functions include:

* `clean_up()`

* `tokenize()`

* `stem_and_lemmatize()`

* `remove_stopwords()`

Create a new column called `text_processed` in the dataframe to contain the processed data. At the end, your `text_processed` column should contain lists of word tokens that are cleaned up. Your data should look like below:

![Processed Data](data-cleaning-results.png)

In [6]:
# your code here


def clean_up(s):
    result = re.sub(r'http?:\/\/.*[\r\n]*', '', s).lower()   
    return re.sub(r'[\W\d]+', ' ', result)

def tokenize(s):
    words = word_tokenize(s)
    return words

lemmatizer = WordNetLemmatizer()
def lemmatize(list_of_words):
    return list(map(lambda s: lemmatizer.lemmatize(s), list_of_words))
    
def remove_stopwords(word_tokens):
    stop_words = set(stopwords.words('english'))
    
    filtered_sentence = [w for w in word_tokens if not w in stop_words] 
    filtered_sentence = [] 

    for w in word_tokens: 
        if w not in stop_words: 
            filtered_sentence.append(w) 
    return filtered_sentence    

In [7]:
test_sentence = "My tailor is rich."

def document_transformer(doc):
    cleaned_doc = clean_up(doc)
    tokens = tokenize(cleaned_doc)
    cleaned_tokens = remove_stopwords(tokens)
    lemmatized_tokens = lemmatize(cleaned_tokens)
    return lemmatized_tokens

print(document_transformer(test_sentence))

['tailor', 'rich']


### Creating Bag of Words

The purpose of this step is to create a [bag of words](https://en.wikipedia.org/wiki/Bag-of-words_model) from the processed data. The bag of words contains all the unique words in your whole text body (a.k.a. *corpus*) with the number of occurrence of each word. It will allow you to understand which words are the most important features across the whole corpus.

Also, you can imagine you will have a massive set of words. The less important words (i.e. those of very low number of occurrence) do not contribute much to the sentiment. Therefore, you only need to use the most important words to build your feature set in the next step. In our case, we will use the top 5,000 words with the highest frequency to build the features.

In the cell below, combine all the words in `text_processed` and calculate the frequency distribution of all words. A convenient library to calculate the term frequency distribution is NLTK's `FreqDist` class ([documentation](https://www.nltk.org/api/nltk.html#module-nltk.probability)). Then select the top 5,000 words from the frequency distribution.

In [8]:
from sklearn.feature_extraction.text import CountVectorizer

corpus = ['This is the first document.',
'This document is the second document.',
'And this is the third one.',
'Is this the first document?']

count_vect = CountVectorizer(tokenizer=document_transformer)

In [9]:
counts = count_vect.fit_transform(corpus)
print(counts.shape)
print(counts.toarray())
print(count_vect.get_feature_names())

(4, 5)
[[1 1 0 0 0]
 [2 0 0 1 0]
 [0 0 1 0 1]
 [1 1 0 0 0]]
['document', 'first', 'one', 'second', 'third']


In [10]:
from sklearn.feature_extraction.text import TfidfTransformer

tfidf_transf = TfidfTransformer()

In [11]:
weights = tfidf_transf.fit_transform(counts)
print(weights.toarray())
print(count_vect.get_feature_names())

[[0.62922751 0.77722116 0.         0.         0.        ]
 [0.78722298 0.         0.         0.61666846 0.        ]
 [0.         0.         0.70710678 0.         0.70710678]
 [0.62922751 0.77722116 0.         0.         0.        ]]
['document', 'first', 'one', 'second', 'third']


In [12]:
emotions.text

365788     arggggggh. I really wanted to go see paramore ...
809004                                       @lukask Thanks 
604206     accidentally slept through her alarm and feels...
399857     Why don't I get to go everywhere you go. I'm a...
611313     working with tweetdeck. loving the multiple ac...
                                 ...                        
874803            @HEYerinlee haha nvm yea i remeber it now 
1449586    Inside the movie house! Freezing. Just had lun...
1202328    GETTING READY TO WATCH MTV MOVIE AWARDS!!! GOO...
122956     @xcarcrashhearts i'm having the same dilema, n...
1541197                   at home enjoying another nice day 
Name: text, Length: 20000, dtype: object

In [13]:
emotions = emotions.reset_index()

In [14]:
count_vect = CountVectorizer(tokenizer=document_transformer, min_df=0.005)
corpus = emotions.text
counts = count_vect.fit_transform(corpus)
tfidf_transf = TfidfTransformer()
weights = tfidf_transf.fit_transform(counts)

In [15]:
weights_df = pd.DataFrame(weights.toarray())
weights_df.columns = count_vect.get_feature_names()

In [16]:
emotions_df = pd.concat([emotions, weights_df], axis=1)

In [17]:
weights_df.columns

Index(['actually', 'almost', 'already', 'also', 'always', 'amazing', 'amp',
       'another', 'around', 'away',
       ...
       'would', 'wow', 'x', 'ya', 'yay', 'yeah', 'year', 'yes', 'yesterday',
       'yet'],
      dtype='object', length=242)

In [18]:
emotions_df.head()

,index,target,id,date,flag,user,text,actually,almost,already,...,would,wow,x,ya,yay,yeah,year,yes,yesterday,yet
0,365788,0,2048632717,Fri Jun 05 15:31:07 PDT 2009,NO_QUERY,MT_gilbert,arggggggh. I really wanted to go see paramore ...,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,809004,4,1469345349,Tue Apr 07 06:05:11 PDT 2009,NO_QUERY,aekins,@lukask Thanks,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,604206,0,2221879520,Thu Jun 18 06:09:02 PDT 2009,NO_QUERY,jenniferohcious,accidentally slept through her alarm and feels...,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,399857,0,2057262646,Sat Jun 06 12:40:29 PDT 2009,NO_QUERY,indy_yu,Why don't I get to go everywhere you go. I'm a...,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,611313,0,2224523224,Thu Jun 18 09:36:24 PDT 2009,NO_QUERY,tinywook,working with tweetdeck. loving the multiple ac...,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [19]:
emotions_df.drop(["id", "date", "flag", "user", "text"], axis=1, inplace=True)

### Building Features

Now let's build the features. Using the top 5,000 words, create a 2-dimensional matrix to record whether each of those words is contained in each document (tweet). Then you also have an output column to indicate whether the sentiment in each tweet is positive. For example, assuming your bag of words has 5 items (`['one', 'two', 'three', 'four', 'five']`) out of 4 documents (`['A', 'B', 'C', 'D']`), your feature set is essentially:

| Doc | one | two | three | four | five | is_positive |
|---|---|---|---|---|---|---|
| A | True | False | False | True | False | True |
| B | False | False | False | True | True | False |
| C | False | True | False | False | False | True |
| D | True | False | False | False | True | False|

However, because the `nltk.NaiveBayesClassifier.train` class we will use in the next step does not work with Pandas dataframe, the structure of your feature set should be converted to the Python list looking like below:

```python
[
	({
		'one': True,
		'two': False,
		'three': False,
		'four': True,
		'five': False
	}, True),
	({
		'one': False,
		'two': False,
		'three': False,
		'four': True,
		'five': True
	}, False),
	({
		'one': False,
		'two': True,
		'three': False,
		'four': False,
		'five': False
	}, True),
	({
		'one': True,
		'two': False,
		'three': False,
		'four': False,
		'five': True
	}, False)
]
```

To help you in this step, watch the [following video](https://www.youtube.com/watch?v=-vVskDsHcVc) to learn how to build the feature set with Python and NLTK. The source code in this video can be found [here](https://pythonprogramming.net/words-as-features-nltk-tutorial/).

[![Building Features](building-features.jpg)](https://www.youtube.com/watch?v=-vVskDsHcVc)

In [20]:
X, y = emotions_df.drop(["target"], axis=1), emotions_df.target

In [21]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [22]:
y_train.value_counts()

0    8014
4    7986
Name: target, dtype: int64

In [23]:
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB().fit(X_train, y_train)

In [24]:
# your code here

### Building and Traininng Naive Bayes Model

In this step you will split your feature set into a training and a test set. Then you will create a Bayes classifier instance using `nltk.NaiveBayesClassifier.train` ([example](https://www.nltk.org/book/ch06.html)) to train with the training dataset.

After training the model, call `classifier.show_most_informative_features()` to inspect the most important features. The output will look like:

```
Most Informative Features
	    snow = True            False : True   =     34.3 : 1.0
	  easter = True            False : True   =     26.2 : 1.0
	 headach = True            False : True   =     20.9 : 1.0
	    argh = True            False : True   =     17.6 : 1.0
	unfortun = True            False : True   =     16.9 : 1.0
	    jona = True             True : False  =     16.2 : 1.0
	     ach = True            False : True   =     14.9 : 1.0
	     sad = True            False : True   =     13.0 : 1.0
	  parent = True            False : True   =     12.9 : 1.0
	  spring = True            False : True   =     12.7 : 1.0
```

The [following video](https://www.youtube.com/watch?v=rISOsUaTrO4) will help you complete this step. The source code in this video can be found [here](https://pythonprogramming.net/naive-bayes-classifier-nltk-tutorial/).

[![Building and Training NB](nb-model-building.jpg)](https://www.youtube.com/watch?v=rISOsUaTrO4)

In [25]:
# your code here: classifier.show_most_informative_features()

### Testing Naive Bayes Model

Now we'll test our classifier with the test dataset. This is done by calling `nltk.classify.accuracy(classifier, test)`.

As mentioned in one of the tutorial videos, a Naive Bayes model is considered OK if your accuracy score is over 0.6. If your accuracy score is over 0.7, you've done a great job!

In [26]:
# your code here

In [27]:
y_pred = clf.predict(X_test)

In [28]:
from sklearn import metrics
print(metrics.classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.93      0.78      0.85      1963
           4       0.82      0.94      0.88      2037

    accuracy                           0.87      4000
   macro avg       0.88      0.86      0.86      4000
weighted avg       0.87      0.87      0.86      4000



## Bonus Question 1: Improve Model Performance

If you are still not exhausted so far and want to dig deeper, try to improve your classifier performance. There are many aspects you can dig into, for example:

* Improve stemming and lemmatization. Inspect your bag of words and the most important features. Are there any words you should furuther remove from analysis? You can append these words to further remove to the stop words list.

* Remember we only used the top 5,000 features to build model? Try using different numbers of top features. The bottom line is to use as few features as you can without compromising your model performance. The fewer features you select into your model, the faster your model is trained. Then you can use a larger sample size to improve your model accuracy score.

In [29]:
# your code here

## Bonus Question 2: Machine Learning Pipeline

In a new Jupyter Notebook, combine all your codes into a function (or a class). Your new function will execute the complete machine learning pipeline job by receiving the dataset location and output the classifier. This will allow you to use your function to predict the sentiment of any tweet in real time. 

In [30]:
# your code here

## Bonus Question 3: Apache Spark

If you have completed the Apache Spark advanced topic lab, what you can do is to migrate your pipeline from local to a Databricks Notebook. Share your notebook with your instructor and classmates to show off your achievements!

In [31]:
# your code here